<a href="https://colab.research.google.com/github/kolessovva/python-ai-Albina-Kolesova/blob/main/notebooks/week2b_read_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Week 2: Data Analysis — Чтение и проверка данных

**Цель**: Научиться читать CSV-файлы из репозитория GitHub в Google Colab и выполнять базовую проверку данных с помощью pandas.

**Данные:**
- [`stars_constellation_starTypeLabel.csv`](https://github.com/kolessovva/python-ai-Albina-Kolesova/blob/main/data/stars_constellation_starTypeLabel.csv) — информация о звёздах: название, созвездие, тип звезды
- [`constellations_area_abbreviation_neighborConstellationsLabel.csv`](https://github.com/kolessovva/python-ai-Albina-Kolesova/blob/main/data/constellations_area_abbreviation_neighborConstellationsLabel.csv) — информация о созвездиях: площадь, аббревиатура, соседние созвездия

**Что мы делаем:**
1. Клонируем ваш репозиторий GitHub в Colab
2. Читаем CSV-файлы в pandas DataFrame
3. Очищаем и переименовываем столбцы для удобства анализа
4. Смотрим структуру данных и делаем быструю валидацию

## 🐱 [1] Клонируем репозиторий курса в Colab

In [2]:
import os

# Код для защиты от вложенных папок исправлен вручную
# Укажите имя папки, в которую клонируем (можно изменить)
repo_name = "python-ai-Albina-Kolesova"

# 🔧 Возвращаемся в корень Colab, чтобы проверка пути работала корректно
%cd /content

if not os.path.exists(repo_name):
    !git clone -q https://github.com/kolessovva/python-ai-Albina-Kolesova.git

%cd {repo_name}

print(f"✅ Репозиторий готов, теперь мы работаем внутри папки {repo_name}")

/content/python-ai-Albina-Kolesova/python-ai-Albina-Kolesova
✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Albina-Kolesova


## 📥 [2A] Простое чтение CSV-файлов в pandas

Сначала просто прочитаем оба CSV-файла в объекты `DataFrame`, без каких‑либо изменений.

После этого мы узнаем, сколько строк загружено в каждый датасет.

In [ ]:
# 📥 Шаг 2. Читаем ваши данные в pandas

import pandas as pd

# Пути к файлам
stars_file = "data/stars_constellation_starTypeLabel.csv"
constellations_file = "data/constellations_area_abbreviation_neighborConstellationsLabel.csv"

# Читаем CSV-файлы
df_stars = pd.read_csv(stars_file)
df_constellations = pd.read_csv(constellations_file)

print(f"🌟 Звёзды: {df_stars.shape[0]} строк, {df_stars.shape[1]} столбцов")
print(f"✨ Созвездия: {df_constellations.shape[0]} строк, {df_constellations.shape[1]} столбцов")

🌟 Звёзды: 2078 строк, 5 столбцов
✨ Созвездия: 527 строк, 5 столбцов


## 🧹 [2B] Очистка и переименование столбцов

В исходных CSV-файлах есть **технические столбцы** с URI Викиданных, которые полезны для ссылочной целостности, но мешают простому анализу:

- Столбцы `star` и `constellation` с URL (например, `http://www.wikidata.org/entity/Q...`) — **сохраняем их для отладки**, но переименуем в `star_URL` и `constellation_URL`.
- Столбцы с суффиксом `Label` (`starLabel`, `constellationLabel`, `starTypeLabel`, `neighborConstellationsLabel`) содержат читаемые названия.

В этом шаге мы:
- переименуем столбцы с URL Викиданных (`star` → `star_URL`, `constellation` → `constellation_URL`);
- переименуем `*Label`-столбцы в более короткие имена: `starLabel → star`, `constellationLabel → constellation`, `starTypeLabel → star_type`, `neighborConstellationsLabel → neighbors`;
- приведём числовые столбцы (`area`) к типу `int`;
- убедимся, что данные готовы к объединению и анализу.

При приведении к числам мы используем:
- `pd.to_numeric(..., errors="coerce")` — преобразует значения в числа, некорректные → `NaN`;
- `fillna(0)` — заменяет пропуски на 0;
- `astype(int)` — переводит столбец к целочисленному типу.

> ⚠️ **Важно:** если в ваших данных есть столбцы с URI Викиданных и суффиксом `*Label`, этот шаг обязателен, чтобы получить аккуратные таблички для анализа. Столбцы `*_URL` пригодятся, если нужно будет быстро перейти к оригинальной записи в Викиданных.

In [ ]:
# 🧹 Шаг 2B. Очистка и переименование столбцов

# ============================================================
# 1) df_stars: информация о звёздах
# ============================================================
if "starLabel" in df_stars.columns:  # Проверка: нужна ли очистка?
    df_stars = df_stars.rename(columns={
        "star": "star_URL",                    # ← технический URL
        "starLabel": "star",                   # ← читаемое название звезды
        "constellation": "constellation_URL",  # ← URL созвездия
        "constellationLabel": "constellation", # ← название созвездия
        "starTypeLabel": "star_type",          # ← тип звезды
    })
    print("✅ df_stars очищен и переименован")
else:
    print("⏭️ df_stars уже очищен, пропускаем")

# ============================================================
# 2) df_constellations: информация о созвездиях
# ============================================================
if "constellationLabel" in df_constellations.columns:  # Проверка: нужна ли очистка?
    df_constellations = df_constellations.rename(columns={
        "constellation": "constellation_URL",          # ← технический URL
        "constellationLabel": "constellation",         # ← читаемое название
        "neighborConstellationsLabel": "neighbors",    # ← соседние созвездия
        # area и abbreviation оставляем как есть
    })

    # Приводим площадь к числовому типу
    df_constellations["area"] = pd.to_numeric(
        df_constellations["area"], errors="coerce"
    ).fillna(0).astype(int)

    # abbreviation уже строка, ничего не меняем
    print("✅ df_constellations очищен и переименован")
else:
    print("⏭️ df_constellations уже очищен, пропускаем")

# ============================================================
# 🔎 Быстрая проверка результата
# ============================================================
print("\n📋 Столбцы после очистки:")
print(f"df_stars: {list(df_stars.columns)}")
print(f"df_constellations: {list(df_constellations.columns)}")

print("\n✅ Данные готовы к анализу и объединению!")

✅ df_stars очищен и переименован
✅ df_constellations очищен и переименован

📋 Столбцы после очистки:
df_stars: ['star_URL', 'star', 'constellation_URL', 'constellation', 'star_type']
df_constellations: ['constellation_URL', 'constellation', 'area', 'abbreviation', 'neighbors']

✅ Данные готовы к анализу и объединению!


## 🔍 [3] Обзор данных: структура и первые строки

Сделаем короткий обзор обоих DataFrame:

- посмотрим размер таблицы (`shape`);
- выведем список столбцов;
- посмотрим первые несколько строк;
- дополнительно посчитаем базовую статистику по площади созвездий (`area`).

Для удобства напишем маленькую функцию `show_info(df, name)`, чтобы не повторять один и тот же код два раза.

In [ ]:
def show_info(df, name, n=5):
    """Краткий обзор DataFrame: имя, размер, список столбцов и первые строки."""
    print(f"\n📊 {name}")
    print("Размер:", df.shape)
    print("Столбцы:", ", ".join(df.columns))
    print("\nПервые строки:")
    print(df.head(n))

# 🔍 Шаг 3. Обзор данных

show_info(df_stars, "Звёзды: названия, созвездия и типы (df_stars)")
show_info(df_constellations, "Созвездия: площадь, аббревиатуры и соседи (df_constellations)")

In [ ]:
# То же самое, но в миллионах долларов (млн $)
# stats_millions = (df_cost["capital_cost"] / 1e6).describe().round(2)
# print("\n📈 Статистика по бюджету в млн $:")
# print(stats_millions)

## ✅ [4] Быстрая проверка и валидация данных

Здесь мы посмотрим:

- сколько **уникальных** звёзд, созвездий и типов звёзд есть в данных;
- **какие созвездия встречаются чаще всего** (Топ‑5 по числу звёзд);
- **какие типы звёзд самые популярные** (Топ‑10 по числу записей);
- **какие аббревиатуры и соседние созвездия присутствуют** в данных о созвездиях.

Функция `value_counts()`:
- считает, сколько раз каждое значение встречается в столбце;
- сортирует результаты по убыванию.

Метод `.head()` берёт первые N строк, поэтому
`df_stars["constellation"].value_counts().head()` даёт **Топ‑5 созвездий по числу звёзд**.

In [ ]:
# ✅ Шаг 4. Быстрая проверка и валидация данных

print("🔍 Быстрая проверка данных")

# ============================================================
# Датасет 1: звёзды, созвездия, типы звёзд
# ============================================================
print("\n📊 Датасет: Звёзды и их характеристики")
print("Уникальных звёзд в df_stars:", df_stars["star"].nunique())
print("Уникальных созвездий:", df_stars["constellation"].nunique())
print("Уникальных типов звёзд:", df_stars["star_type"].nunique())

print("\nТоп-5 созвездий по числу звёзд:")
print(df_stars["constellation"].value_counts().head())

print("\nТоп-10 типов звёзд:")
print(df_stars["star_type"].value_counts().head(10))

# ============================================================
# Датасет 2: созвездия (площадь, аббревиатуры, соседи)
# ============================================================
print("\n📊 Датасет: Созвездия и их характеристики")
print("Уникальных созвездий в df_constellations:", df_constellations["constellation"].nunique())
print("Уникальных аббревиатур:", df_constellations["abbreviation"].nunique())
print("Диапазон площадей (кв. градусы):", df_constellations["area"].min(), "—", df_constellations["area"].max())

print("\nАббревиатуры созвездий (первые 15):")
print(df_constellations["abbreviation"].value_counts().head(15))

print("\nПримеры соседних созвездий:")
print(df_constellations["neighbors"].value_counts().head(10))

print("\nСтатистика по площадям созвездий:")
print(df_constellations["area"].describe())

## 📝 Summary

**Что мы сделали в этом ноутбуке (Week 2):**

- ✅ Клонировали репозиторий GitHub в Colab
- ✅ Прочитали 2 CSV-файла из `data/examples/`
- ✅ Удалили URL Wikidata и переименовали столбцы (`*Label → короткие имена`)
- ✅ Проверили структуру данных (размер, столбцы, первые строки)
- ✅ Выполнили быструю валидацию:
  - количество уникальных фильмов, стран, жанров
  - диапазоны значений
  - топ стран и жанров по числу записей
  - типы оценок и результатов

Теперь у нас есть **аккуратные, проверенные таблицы**, с которыми удобно работать дальше.

В отдельном ноутбуке для следующей недели мы будем использовать **те же данные** для:
- более сложного анализа (группировки, фильтрация),
- и построения визуализаций (графики и диаграммы). 🎨